In [12]:
import pandas as pd
import numpy as np

#Graphes
import igraph as ig
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ipysigma import Sigma
import matplotlib.pyplot as plt
import spacy
#Parcours de documents et tokenisation
import os
from tqdm import tqdm
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import csv
#LDA
import gensim
from gensim import corpora
import logging
import time
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Topic Modeling par année 

#### Etape 1 : Prendre des courts extraits aléatoires de chaque séance 

In [13]:
# Fonction pour importer les documents et les regrouper par années
def ouverture_documents(chemin_dossier):
    liste_fichiers = os.listdir(chemin_dossier)
    documents_par_annee = {}
    
    for fichier in tqdm(liste_fichiers):
        with open(os.path.join(chemin_dossier, fichier), 'r', encoding="utf-8") as texte:
            seance = texte.read()
        
        # Extraction de l'année à partir du nom du fichier
        annee = fichier[:4]
        
        # Ajout du contenu du document à la liste de l'année correspondante
        if annee not in documents_par_annee:
            documents_par_annee[annee] = []
        documents_par_annee[annee].append(seance)
    
    return documents_par_annee




In [14]:
chemin_dossier = "C:/Users/portable_laura/Documents/coursM1/Mémoire/Sources/ocr_sorted"
docus_par_annee = ouverture_documents(chemin_dossier)

100%|██████████| 7461/7461 [02:37<00:00, 47.48it/s]


In [29]:
import random
def extraits_aleatoires(documents_par_annee, longueur_souhaitee=15000):
    extraits_par_annee = {}

    for annee, documents in tqdm(documents_par_annee.items()):
        extraits_par_annee[annee] = []
        for document in documents:
            longueur_chaine = len(document)
            if longueur_chaine <= longueur_souhaitee:
                extraits_par_annee[annee].append(document)
            else:
                #prend un index de départ aléatoire entre le début de la chaîne et le dernier index permettant d'obtenir une chaine de la longueur souhaitée
                index = random.randint(0, longueur_chaine - longueur_souhaitee)
                extraits_par_annee[annee].append(document[index:index + longueur_souhaitee])
    
    return extraits_par_annee

In [30]:
extraits_docus = extraits_aleatoires(docus_par_annee)
extraits_docus.keys()

100%|██████████| 60/60 [00:07<00:00,  7.66it/s]


dict_keys(['1881', '1882', '1883', '1884', '1885', '1886', '1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940'])

#### Etape 2 : Tokeniser chaque année 

In [33]:
nltk.download('stopwords')
nltk.download('punkt')
stopwords_fr = stopwords.words('french')


#Remplir manuellement les mots qui semblent pertinents dans le contexte des débats parlementaires de la troisième république
stopwords_fr.extend([
    'monsieur', 'madame',"présidences",'présidents', 'président',"secrétaires","présidente","voix","bureau",'secrétaire', 'ministre', 'rapporteur', 'commission', 'député', 'sénateur', 'assemblée', 'sénat', 'parlement', 'république', 'projet', 'loi', 'amendement', 
    'article', 'alinéa', 'vote', 'scrutin', 'débat', 'intervention', 'orateur', 'gouvernement', 'commission', 
    'question', 'réponse', 'session', 'ordre', 'jour', 'discussion', 'texte', 'examen', 'proposition', 'hémicycle', 
    'session', 'majorité', 'minorité', 'opposition', 'camarade', 'camarades', 'débat', 'interventions', 'présidence', 
    'directeur', 'décision', 'pouvoir', 'réunion', 'groupes', 'parlementaire', 'parlementaires', 'politique', 
    'politiques', 'publique', 'public', 'publique', 'privé', 'privée', 'national', 'nationale', 'nation', 'nations', 
    'citoyen', 'citoyenne', 'citoyens', 'citoyennes', 'mesdames', 'messieurs', 'collègue', 'collègues', 'disposition', 
    'dispositions',"comme","cette","être","an","","dit","sée","a","rapp","si","si","do","dit","a","bien","plus","cette","mm","dont","parce","e","très","si"
])


with open("C:/Users/portable_laura/Documents/coursM1/Mémoire/Code/stop_words_french.txt","r",encoding="utf-8") as fichier:
    stopwords_ajout = fichier.read()
    
stopwords_ajout = stopwords_ajout.replace('\n', ' ')
    
liste_stopwords = stopwords_ajout.split()

stopwords_fr.extend(liste_stopwords)

print(stopwords_fr)
print(len(stopwords_fr))

['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aur

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\portable_laura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\portable_laura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [34]:
#Preprocessing des documents - fonction
def preprocess(doc_texte):
    doc_texte = doc_texte.lower()
    doc_texte = re.sub(r'[^\w\sÀ-ÿ]', ' ', doc_texte)  # Utilisation de la plage Unicode pour inclure les caractères français
    doc_texte = re.sub(r'\d+', ' ', doc_texte)
    words = word_tokenize(doc_texte, language='french')  # Tokenization pour le français
    
    words = [word for word in words if word not in stopwords_fr]  # Utilisation de mots vides en français, cette liste est à modifier et a agrémenter des mots vides correspondant au corpus
    return words

In [35]:
def preprocess_extraits(extraits_par_annee):
    preprocessed_extraits = {}

    for annee, extraits in extraits_par_annee.items():
        preprocessed_extraits[annee] = []
        for extrait in tqdm(extraits, desc=f"Préprocessing de l'année {annee}", leave=False):
            preprocessed_extraits[annee].append(preprocess(extrait))
    
    return preprocessed_extraits


In [36]:
dico_extraits_preprocess = preprocess_extraits(extraits_docus)

In [37]:
import json
#On enregistre le dictionnaire dans un fichier json afin de pouvoir l'exploiter à nouveau par la suite

fichier_json = "C:/Users/portable_laura/Documents/coursM1/Mémoire/Code/dico_extraits_preprocess_15000.json"

with open(fichier_json, 'w', encoding='utf-8') as fichier:
    json.dump(dico_extraits_preprocess, fichier, ensure_ascii=False, indent=4)

#### Etape 3 : Déterminer les sujets pour chaque année ou tranche d'années

In [38]:

def create_lda_and_save_topics(dico, filename):
    all_topics = []

    for key, docs in dico.items():
        dictionary = corpora.Dictionary(docs)
        corpus = [dictionary.doc2bow(doc) for doc in tqdm(docs, desc=f"Création du corpus pour {key}")]

        num_topics = 20
        passes = 20

        lda_model = gensim.models.LdaModel(
            corpus, 
            num_topics=num_topics, 
            id2word=dictionary, 
            passes=passes,
        )

        for idx, topic in lda_model.print_topics(-1):
            all_topics.append([key, idx, topic])

    topics_df = pd.DataFrame(all_topics, columns=['Key', 'Topic Index', 'Words'])
    topics_df.to_csv(filename, index=False)

create_lda_and_save_topics(dico_extraits_preprocess, 'C:/Users/portable_laura/Documents/coursM1/Mémoire/Code/lda_topics_test_avec_15000.csv')


Création du corpus pour 1940: 100%|██████████| 53/53 [00:00<00:00, 1039.55it/s]


In [39]:
nom_fichier = 'C:/Users/portable_laura/Documents/coursM1/Mémoire/Code/lda_topics_test_avec_15000.csv'

pd.set_option('display.max_colwidth', None)
topics_df = pd.read_csv(nom_fichier)

topics_df.head(40)


,Key,Topic Index,Words
0,1881,0,"0.008*""chambre"" + 0.006*""droit"" + 0.006*""demande"" + 0.003*""gauche"" + 0.003*""service"" + 0.003*""fr"" + 0.003*""heure"" + 0.003*""lieu"" + 0.003*""heures"" + 0.003*""ans"""
1,1881,1,"0.006*""chambre"" + 0.005*""gauche"" + 0.005*""demande"" + 0.005*""fr"" + 0.004*""droit"" + 0.004*""france"" + 0.003*""pays"" + 0.003*""ministère"" + 0.003*""faut"" + 0.003*""guerre"""
2,1881,2,"0.019*""dépôt"" + 0.018*""fer"" + 0.014*""chemin"" + 0.012*""objet"" + 0.008*""publ"" + 0.006*""utilité"" + 0.006*""adopt"" + 0.006*""ligne"" + 0.005*""déclarat"" + 0.005*""saint"""
3,1881,3,"0.007*""chambre"" + 0.005*""demande"" + 0.004*""officiers"" + 0.004*""gauche"" + 0.004*""armée"" + 0.003*""fr"" + 0.002*""droit"" + 0.002*""moment"" + 0.002*""chabrié"" + 0.002*""freppel"""
4,1881,4,"0.011*""nombre"" + 0.010*""inscrits"" + 0.010*""exprimés"" + 0.010*""suffrages"" + 0.010*""quart"" + 0.008*""électeurs"" + 0.008*""obtenu"" + 0.008*""bulletins"" + 0.007*""protestation"" + 0.007*""votants"""
5,1881,5,"0.006*""ouvriers"" + 0.004*""droit"" + 0.004*""chambre"" + 0.004*""provisoire"" + 0.003*""outrage"" + 0.003*""syndicales"" + 0.003*""vice"" + 0.003*""délit"" + 0.003*""associations"" + 0.002*""beauquier"""
6,1881,6,"0.005*""cuirassés"" + 0.003*""nombre"" + 0.003*""police"" + 0.003*""gauche"" + 0.003*""prieu"" + 0.003*""point"" + 0.003*""service"" + 0.003*""budget"" + 0.002*""etat"" + 0.002*""situation"""
7,1881,7,"0.011*""chambre"" + 0.006*""gauche"" + 0.005*""droit"" + 0.005*""demande"" + 0.004*""budget"" + 0.003*""membres"" + 0.002*""etat"" + 0.002*""lieu"" + 0.002*""syndicats"" + 0.002*""haentjens"""
8,1881,8,"0.008*""adopté"" + 0.007*""chambre"" + 0.007*""art"" + 0.006*""travaux"" + 0.006*""budget"" + 0.005*""millions"" + 0.004*""etat"" + 0.004*""dépenses"" + 0.004*""droit"" + 0.003*""fr"""
9,1881,9,"0.007*""électeurs"" + 0.005*""provost"" + 0.005*""launay"" + 0.004*""nord"" + 0.004*""tracé"" + 0.003*""gauche"" + 0.003*""arrondissement"" + 0.003*""chambre"" + 0.003*""even"" + 0.003*""préfet"""
